In [2]:
import numpy as np
import pandas as pd
from scipy.special import softmax
import csv
import urllib.request
import ssl
import certifi
import os

In [3]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

#tokenizer = AutoTokenizer.from_pretrained(MODEL)
#model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(save_directory='../models/' + MODEL)
#tokenizer.save_pretrained(save_directory='../models/' + MODEL)

tokenizer = AutoTokenizer.from_pretrained('../models/' + MODEL, local_files_only=True)
model = TFAutoModelForSequenceClassification.from_pretrained('../models/' + MODEL, local_files_only=True)


2022-02-06 23:21:06.135254: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at ../models/cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [4]:
# download label mapping
labels=[]
task='sentiment'
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link, context=ssl.create_default_context(cafile=certifi.where())) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [5]:
# # TF Examples
#Yeah exactly, dxy is pumping and every thing is still green.. SPX and DJI and BTC.. struggling yeah, but not dumping though, given that bulls and bears are calling for the 38k.. 💁🏻‍♂️ So where do you see us go from here? In the next weeks and until the end of year?
#"I think bitcoin is really on the verge of getting broad acceptance by conventional finance people"
#neu1 = "WHY CHINA’S RENEWED HARSH CRACKDOWN ON CRYPTO ISN’T STIFLING BITCOIN — AT LEAST NOT YET #bitcoin #btc"
#text1 = "If Bitcoin was able to capture a $1T market cap, it's not even impossible at this point. It might take time. But there is a chance."
positive = "If SEC approves the one of the ETF’s , this will allow people and instutitional investors to buy bitcoin easier."
#neg1 = "yesterday $btc suffered the largest percentage drop in a single day since june 21st📉, falling by almost $6,000! are you buying the dip or do you think we'll see more downside? bet on the price of (⬆️ or ⬇️) with up to x1000 leverage👉:"
#neg2 = "problems continue to plague el salvador's bitcoin rollout"
def predict(text):
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(encoded_input)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    print(scores)
    print(labels[0])
    print(labels[1])
    print(labels[2])
    """ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")"""
predict(positive)

[0.00341765 0.31853524 0.67804724]
negative
neutral
positive


In [6]:
#load preprocessed dataset
data = pd.read_csv('../data/processed/bitcoin_tweets_result_processed_20210101-20210930.csv', index_col=0)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
print(data.shape)
data.head()
data.info()

(19936535, 13)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19936535 entries, 0 to 20357615
Data columns (total 13 columns):
 #   Column             Dtype  
---  ------             -----  
 0   id                 int64  
 1   date               object 
 2   text               object 
 3   hashtags           object 
 4   replyCount         float64
 5   retweetCount       float64
 6   likeCount          float64
 7   userName           object 
 8   userFollowerCount  float64
 9   userFavCount       float64
 10  userFriendCount    float64
 11  dateTime           object 
 12  textLength         int64  
dtypes: float64(6), int64(2), object(5)
memory usage: 2.1+ GB


In [31]:
#select a slice of the dataset for prediction, based on dateTime
#data = data.drop(['Unnamed: 0'])

sData = data[(data['dateTime'] >= '2021-07-20') & (data['dateTime'] < '2021-08-10')]

sData = sData.dropna(subset=['text'])

print(sData[sData['text'].map(type)== str].shape) 

print('the number of selected texts:', sData.shape[0])

print('estimated completion time:', sData.shape[0] * 0.16 / 60 / 60, 'hours')

(1349440, 13)
the number of selected texts: 1349440
estimated completion time: 59.97511111111111 hours


In [32]:
#apply prediction on texts
def predict(text):
    try:
        encoded_input = tokenizer(text, return_tensors='tf')
        output = model(encoded_input)
        scores = output[0][0].numpy()
        scores = softmax(scores)
        negative = np.round(float(scores[0]), 4)
        neutral = np.round(float(scores[1]), 4)
        positive = np.round(float(scores[2]), 4)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        prediction = labels[ranking[0]]
        #score = scores[ranking[0]]
        #confidence = np.round(float(score), 4)
        return pd.Series([prediction, negative, neutral, positive])
    except:
        print('error:', text)
        return pd.Series(['unknown', 0,0,0])

#btcdata['sentiment'] = btcdata['text'].apply(lambda x: x.lower())
sData[['prediction','negative', 'neutral', 'positive']] = sData['text'].apply(lambda x: predict(x))

error: jfc!! im like pavlov's dog now, every time i hear "pump it up" ✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾✊🏾


In [29]:
sData.head()

,id,date,text,hashtags,replyCount,retweetCount,likeCount,userName,userFollowerCount,userFavCount,userFriendCount,dateTime,textLength,prediction,negative,neutral,positive
2938669,1428507066147262464,2021-08-19 23:59:59+00:00,"finish this sentence: this weekend, will reach...",['Bitcoin'],20.0,1.0,27.0,Okcoin,139336.0,15535.0,939.0,2021-08-19 23:59:59+00:00,51,neutral,0.0425,0.8850,0.0725
2938670,1428507059272798210,2021-08-19 23:59:57+00:00,"if anyone knows me, they know i'm as bullish a...",['bitcoin'],0.0,0.0,0.0,kluza,313.0,2957.0,998.0,2021-08-19 23:59:57+00:00,269,neutral,0.3460,0.5445,0.1095
2938671,1428507058807189511,2021-08-19 23:59:57+00:00,49.5% in this optimal portfolio and 50.5% in t...,"['Investing', 'crypto', 'cash', 'Bitcoin', 'ri...",0.0,0.0,0.0,CryptoWatchBot,2993.0,215.0,2891.0,2021-08-19 23:59:57+00:00,140,neutral,0.0423,0.6954,0.2624
2938672,1428507033649786882,2021-08-19 23:59:51+00:00,🚨 +5% profit of $mana-usd in less than 9 hours...,"['MANA', 'BTC', 'Bitcoin', 'Crypto', 'cryptocu...",0.0,0.0,0.0,torshis,1586.0,121.0,0.0,2021-08-19 23:59:51+00:00,127,neutral,0.0093,0.5193,0.4714
2938673,1428507028067176453,2021-08-19 23:59:50+00:00,bitcoin.,NaN,0.0,0.0,0.0,Binary_Miner,1004.0,16818.0,811.0,2021-08-19 23:59:50+00:00,8,neutral,0.0811,0.7214,0.1976


In [30]:
filename = '../data/interim/bitcoin_all_tweets_sentiments_20210101-20210930.csv'
file_exists = os.path.isfile(filename)

#Open/create a file to append data to
#csvFile = open(filename, 'a', newline='', encoding='utf8')

#Use csv writer
#csvWriter = csv.writer(csvFile)
if not file_exists:
    sData.to_csv(filename, mode='w')
sData.to_csv(filename, mode='a', header=False)
#csvFile.close()